# Predicting the conversion MCI -> AD

In [2]:
import pandas as pd
from skrebate import ReliefF
import numpy as np
from sklearn.feature_selection import mutual_info_classif
from mrmr import mrmr_classif
from ITMO_FS.filters.multivariate import CMIM
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, accuracy_score, roc_auc_score, confusion_matrix
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.naive_bayes import GaussianNB, ComplementNB
from sklearn import tree
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE, SMOTENC
from imblearn.pipeline import Pipeline as ImbPipeline 
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn import set_config
from sklearn.preprocessing import OneHotEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.inspection import permutation_importance
import shap
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression, PLSCanonical
#import FSM

In [5]:
import sys
import logging

nblog = open("ensemble_prediction.log", "a+")
sys.stdout.echo = nblog
sys.stderr.echo = nblog

get_ipython().log.handlers[0].stream = nblog
get_ipython().log.setLevel(logging.INFO)

%autosave 5

Autosaving every 5 seconds


In [6]:
tw = 5
dir ='lisbon'
df = pd.read_csv('../data/'+dir+'/conversion_ad/{}tw_no_norm.csv'.format(tw))
all_features = df.copy()
y = df['Evolution'].copy()
all_features.drop(columns = ['Code','Group', 'BBA', 'CSFdate', 'comentarios', 'Conversion', 'data', 'tempofollowup', 'Evolution'], inplace = True) # drop unwanted columns
all_features['sexo'].replace({'M' : 0, 'F': 1}, inplace = True)
numerical_features = [feature for feature in all_features.columns if feature not in ['sexo', 'Cluster']]
#normalize non normalized columns: idade, CSFdatatTau, CSFdatapTau, CSFdataabeta42, MMSE
#non_normalized_columns = ['idade', 'CSFdatatTau', 'CSFdatapTau', 'CSFdataabeta42', 'MMSE']
#all_features[non_normalized_columns] = (all_features[non_normalized_columns] -all_features[non_normalized_columns].min())/(all_features[non_normalized_columns].max() - all_features[non_normalized_columns].min())
all_features

,idade,sexo,CSFdatatTau,CSFdatapTau,CSFdataabeta42,MMSE,Cluster,NRP2,APOA1,FETUA,...,B4GA1,KV127,NAR3,MYO6,MANBA,SODM,FIBG,CNTP4,HV349,A2AP
0,76,1,618,95,552,27,0,0.000155,0.00331,0.000652,...,0.00184,9.050000e-06,0.000070,0.000059,0.000210,0.000079,0.00122,0.000135,0.000127,0.000954
1,77,0,563,77,349,30,1,0.000020,0.00434,0.000902,...,0.00201,1.460000e-06,0.000161,0.000034,0.001250,0.000043,0.00257,0.000411,0.000096,0.000550
2,72,0,1201,135,399,22,0,0.000031,0.00223,0.000419,...,0.00258,6.540000e-06,0.000165,0.000037,0.000333,0.000297,0.00173,0.000242,0.000077,0.000539
3,61,1,680,104,389,26,1,0.000145,0.00514,0.001170,...,0.00202,2.540000e-05,0.000173,0.000049,0.001120,0.000084,0.00230,0.000868,0.000151,0.000663
4,63,1,200,26,324,22,1,0.000012,0.00313,0.000769,...,0.00154,7.520000e-07,0.000116,0.000372,0.000826,0.000284,0.00124,0.000973,0.000117,0.000697
5,59,1,540,74,504,24,0,0.000032,0.00370,0.000844,...,0.00223,3.090000e-06,0.000169,0.000052,0.000279,0.000098,0.00143,0.000432,0.000141,0.000695
6,74,1,404,68,855,23,1,0.000019,0.00487,0.001050,...,0.00229,9.840000e-06,0.000239,0.000029,0.000466,0.000074,0.00277,0.000298,0.000100,0.001120
7,73,0,770,103,540,23,1,0.000020,0.00445,0.001070,...,0.00146,1.400000e-06,0.000107,0.000049,0.000207,0.000076,0.00243,0.000317,0.000160,0.001130
8,61,1,1080,110,475,25,1,0.000013,0.00443,0.000885,...,0.00125,1.010000e-05,0.000091,0.000029,0.000761,0.000053,0.00235,0.000205,0.000158,0.000899
9,71,0,702,86,558,26,0,0.000013,0.00451,0.000796,...,0.00193,8.910000e-06,0.000169,0.000038,0.000262,0.000070,0.00231,0.000928,0.000099,0.000514


**Feature selection ensemble** 

FS methods: ReliefF, MIM, CMIM, MRMR, Chi-Squared, LL21 (do not understand the implementation, used rf as the embedded method instead)
SVM-RFE excluded (see telma's paper)

In [22]:
class FS_ensemble(BaseEstimator, TransformerMixin):
    def __init__(self, k):
        self.k = k

    def fs_ensemble(self,x_train, y_train):
        # get rankings of each fs method
        print('FS')
        #relieff
        #reliefFS = ReliefF(n_neighbors = 2, n_features_to_select = len(x_train.columns)) #n_features_to_keep = len(x_train.columns)
        #reliefFS.fit(x_train.values, y_train)
        #relief_rank = np.argsort(np.argsort(-reliefFS.feature_importances_)) # rank of the features according to it importance: the higher the importance the lower the rank

        # mim
        mutual_info = mutual_info_classif(x_train, y_train, random_state = 42)
        mutual_info_rank = np.argsort(np.argsort(-mutual_info))

        # cmim
        #cmimFS = CMIM(np.empty(0), np.array(x_train.columns), X = x_train.values, y = y_train)
        #cmim_rank = np.argsort(np.argsort(-cmimFS))

        #mrmr
        #mrmrFS = mrmr_classif(X=x_train, y=y_train, K=len(x_train.columns), n_jobs = -1, show_progress = False)
        #mrmrFS_rank = [mrmrFS.index(item) for item in x_train.columns]

        # chi-squared
        #chi2_filter = chi2(x_train.values,y_train)
        #chi2_rank = np.argsort(np.argsort(-chi2_filter[0]))

        #anova
        f_values, p_values = f_classif(x_train.values, y_train)
        f_rank = np.argsort(np.argsort(-f_values))

        #VIF
        #vif = [variance_inflation_factor(x_train.values, i) for i in range(len(x_train.columns))]
        #vif_rank = np.argsort(np.argsort(vif))

        #LL21
        #selection = SelectFromModel(LogisticRegression(solver = 'liblinear', penalty='l1'))
        #selection.fit(x_train.values, y_train)
        #ll21_rank = x_train.columns[(selection.get_support())]
        #print(ll21_rank)
        lr = LogisticRegression(penalty='l1', solver='liblinear', random_state = 42)#, l1_ratio=0.5, max_iter=10000, dual = True)
        lr.fit(x_train, y_train)
        lr_importances = np.absolute(lr.coef_[0])
        lr_rank = np.argsort(np.argsort(-lr_importances))
        # Assign the rank len(X.columns) to the zero coefficients


        # permutation importance rf
        rf = RandomForestClassifier(n_estimators=100, max_features = None, random_state=42)
        rf.fit(x_train, y_train)
        #rf_importances = permutation_importance(rf, x_train, y_train, n_repeats=4, random_state=42, n_jobs=-1)
        #rf_importances_mean = rf_importances.importances_mean
        rf_importances = rf.feature_importances_
        rf_rank = np.argsort(np.argsort(-rf_importances))

        # SHAP
        explainer = shap.TreeExplainer(rf)
        # Calculate SHAP values for the train set
        shap_values_train = explainer.shap_values(x_train)
        #print(len(shap_values_train))
        # Mean absolute SHAP values to determine global feature importance
        #shap_importance_train = np.abs(shap_values_train).mean(0)
        shap_importance_train = np.mean([np.abs(val).mean(0) for val in shap_values_train], axis=0)
        #print(shap_importance_train.shape)
        shap_rank = np.argsort(np.argsort(-shap_importance_train))
        #print(len(shap_rank))
        #xgb
        #xgb = XGBClassifier(random_state = 42)
        #xgb.fit(x_train, y_train)
        #xgb_importances = xgb.feature_importances_
        #xgb_rank = np.argsort(np.argsort(-xgb_importances))

        #Rank aggregator: get a final subset in which the rank is the mean of all subsets
        #res = [sum(values) /6 for values in zip(mutual_info_rank + chi2_rank + f_rank+ lr_rank + rf_rank + xgb_rank)] #average position
        res = [sum(values) / len(values) for values in zip(mutual_info_rank, f_rank, lr_rank, rf_rank, shap_rank)]
        ranked_features = dict(zip(x_train.columns, res))
        #print(ranked_features['idade'])
        ranked_features = list(dict(sorted(ranked_features.items(), key=lambda item: item[1])).keys())
        #print(ranked_features)
        return ranked_features[:self.k]

    
    def fit(self, X, y=None):
        # Apply the feature selection function to the training data
        self.selected_features_ = self.fs_ensemble(X, y)
        
        return self
    
    def transform(self, X):
        # Select features based on the result from feature_selector_func
        return X.loc[:, self.selected_features_]
    
    def get_features(self):
        return self.selected_features_

In [8]:
# Custom transformer to return DataFrame after MinMax scaling
class DataFrameMinMaxScaler(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()
        self.columns = []

    def fit(self, X, y=None):
        self.columns = X.columns
        self.scaler.fit(X, y)
        return self

    def transform(self, X):
        self.columns = X.columns
        X_scaled = self.scaler.transform(X)
        # Convert back to DataFrame, keeping original column names
        return pd.DataFrame(X_scaled, columns=X.columns, index=X.index)
        
    def fit_transform(self, X, y=None):
        X_scaled = self.scaler.fit_transform(X)
        return pd.DataFrame(X_scaled, columns=self.columns, index=X.index)

In [13]:
class DynamicSMOTENC(BaseEstimator):
    def __init__(self, sampling_strategy=0.95, random_state=42):
        self.sampling_strategy = sampling_strategy
        self.random_state = random_state
        self.smote = None

    def fit(self, X, y=None):
        
        # Identify categorical feature indices
        self.cat_indices = [i for i, column in enumerate(X.columns) if column in ['sexo', 'Cluster']]
    
        # Choose SMOTENC if there are categorical features, otherwise SMOTE
        if self.cat_indices:
            self.smote = SMOTENC(categorical_features=self.cat_indices, categorical_encoder = OneHotEncoder(sparse_output = False),
                                 sampling_strategy=self.sampling_strategy, k_neighbors=3,
                                 random_state=self.random_state)
            #self.smote.ohe_.set_params({'sparse_output': False})
        else:
            self.smote = SMOTE(sampling_strategy=self.sampling_strategy, k_neighbors=3,
                               random_state=self.random_state)
        
        # Fit the SMOTE/SMOTENC with X and y
        self.smote.fit(X, y)
        return self

    def fit_resample(self, X, y=None):
        self.cat_indices = [i for i, column in enumerate(X.columns) if column in ['sexo', 'Cluster']]
    
        # Choose SMOTENC if there are categorical features, otherwise SMOTE
        if self.cat_indices:
            self.smote = SMOTENC(categorical_features=self.cat_indices, categorical_encoder = OneHotEncoder(sparse_output = False),
                                 sampling_strategy=self.sampling_strategy, k_neighbors=3,
                                 random_state=self.random_state)
            #self.smote.ohe_.set_params({'sparse_output': False})

        else:
            self.smote = SMOTE(sampling_strategy=self.sampling_strategy, k_neighbors=3,
                               random_state=self.random_state)

        # Apply SMOTE/SMOTENC
        X_res, y_res = self.smote.fit_resample(X, y)
        #X_res = pd.DataFrame(X_res, columns=X.columns)
        return X_res, y_res
            


10x5 Cross Validation

In [15]:
#classifiers to evaluate
# Define classifiers and their parameter grids
classifiers = {
    'LogisticRegression': {
        'model': LogisticRegression(solver='liblinear'),
        'params': {
            'classification__C': [0.1, 0.5, 1, 5, 10],
            'classification__class_weight' : [None, 'balanced'],
            'classification__penalty': ['l1', 'l2'], 'classification__random_state' : [42]
        }
    },
    'RandomForestClassifier': {
        'model': RandomForestClassifier(),
        'params': {
            'classification__n_estimators': [10,50, 100, 200],
            'classification__max_depth': [None, 10, 15,20,25],
            'classification__class_weight' : [None, 'balanced'], 'classification__random_state' : [42]
        }
    },
    'XGBgClassifier': {
        'model': XGBClassifier(),
        'params': {
            'classification__n_estimators': [10, 50, 100,300],
            'classification__max_depth': [5,10,15,20],
            'classification__learning_rate': [0.0001, 0.001, 0.01],
            'classification__objective': ['binary:logistic'], 'classification__random_state' : [42]
        }
    },
    'SVC': {
        'model': SVC(),
        'params': {
            'classification__C': [0.1, 0.5, 1, 10],
            'classification__kernel': ['linear','poly','rbf', 'sigmoid'],
            'classification__probability': [True],
            'classification__class_weight' : [None, 'balanced'], 'classification__random_state' : [42]
        }
    },
    'NB': {
        'model': GaussianNB(),
        'params': {'classification__var_smoothing' : [10**-9, 10**-8, 10**-10]}
    },
    'DTClassifier': {
    'model': tree.DecisionTreeClassifier(),
    'params': {'classification__max_depth': [None, 10, 20],
              'classification__class_weight' : [None, 'balanced'],
              'classification__random_state' : [42]}
    }
}

In [15]:
def tn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 0]
def fp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[0, 1]
def fn(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 0]
def tp(y_true, y_pred): return confusion_matrix(y_true, y_pred)[1, 1]


def sens(y_true, y_pred): return tp(y_true, y_pred) / \
    (fn(y_true, y_pred) + tp(y_true, y_pred))


def spec(y_true, y_pred): return tn(y_true, y_pred) / \
    (fp(y_true, y_pred) + tn(y_true, y_pred))


In [17]:
sensitivity_scorer = make_scorer(sens)

# Specificity scorer
specificity_scorer = make_scorer(spec)

# AUC scorer
auc_scorer = make_scorer(roc_auc_score, needs_proba=True)

# accuracy scorer
accuracy_scorer = make_scorer(accuracy_score)

In [ ]:
#simple example
X_train, X_test, y_train, y_test = train_test_split(all_features, y, test_size=0.33, random_state=42)
for k in range(1,len(all_features.columns)):
    print(k)
    selected = FS_ensemble(k = k)
    selected.fit(X_train,y_train)
    train_data = selected.transform(X_train)
    clf = LogisticRegression(random_state=0).fit(train_data, y_train)
    test_data = selected.transform(X_test)
    clf.predict(test_data)
    print(clf.score(test_data, y_test))

In [23]:
# Define the cross-validation procedure (5-fold cross-validation with 10 repetitions)
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=42)
all_scores = []
k_scores = []
# List to store results
results = []
#print(all_features)
set_config(transform_output="pandas")
# Define the ColumnTransformer to apply MinMaxScaler only to numerical columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numerical_features),  # Custom scaler for numerical features
        ('cat', 'passthrough', ['sexo', 'Cluster'])            # Pass categorical columns unchanged
    ], remainder='passthrough', verbose_feature_names_out=False, sparse_threshold=0)     #remainder='passthrough'  # Ensure other columns are passed through if not specified

for k in range(2,len(all_features.columns)):   
    #fs_ensemble = FS_ensemble(k = k) #select subset of features
    #k = 92
    print(k)
    for name, clf in classifiers.items():
        print(name)
        pipeline = ImbPipeline([('scaler', preprocessor),
                                ('feature_selection', FS_ensemble(k = k)), ('SMOTE', DynamicSMOTENC(sampling_strategy=0.8, random_state = 42)),
                                #('pca', PCA()),
                                ('classification', clf['model'])])
        
        #clf['params']['pca__n_components'] = list(range(2, min(k+1, len(all_features)), 5))
        
        # Perform grid search with cross-validation
        grid_search = GridSearchCV(pipeline, clf['params'], cv=cv, scoring={'AUC': auc_scorer, 'Sensitivity': sensitivity_scorer, 'Specificity': specificity_scorer, 'Accuracy': accuracy_scorer}, refit = 'AUC', n_jobs=-1)
        grid_search.fit(all_features, y)
        
        # Get the index of the best model (based on AUC score)
        best_index = grid_search.best_index_
         # Extract the sensitivity and specificity for the best model
        best_sensitivity = grid_search.cv_results_['mean_test_Sensitivity'][best_index]
        best_std_sensitivity = grid_search.cv_results_['std_test_Sensitivity'][best_index]
        best_specificity = grid_search.cv_results_['mean_test_Specificity'][best_index]
        best_std_specificity = grid_search.cv_results_['std_test_Specificity'][best_index]
        best_accuracy = grid_search.cv_results_['mean_test_Accuracy'][best_index]
        best_std_accuracy = grid_search.cv_results_['std_test_Accuracy'][best_index]
        # Get the best estimator (pipeline) from the grid search
        best_model = grid_search.best_estimator_
        # Get the selected features from the best feature selection step
        selected_features = best_model.named_steps['feature_selection']
        selected_features = selected_features.get_features()
        
        # Store results
        results.append({
        'classifier': name,
        'best_params': grid_search.best_params_,
        'best_auc': grid_search.best_score_,
        'best_std_auc' :  grid_search.cv_results_['std_test_AUC'][best_index],
        'best_sensitivity': best_sensitivity,
        'best_std_sensitivity': best_std_sensitivity,
        'best_specificity': best_specificity,
        'best_std_specificity': best_std_specificity,
        'best_accuracy': best_accuracy,
        'best_std_accuracy': best_std_accuracy,
        '#features': k,
        'features': selected_features
        })
        print('auc: ', grid_search.best_score_)

# Find the best classifier based on accuracy
best_result = max(results, key=lambda x: x['best_auc'])
best = f"Best Classifier: {best_result['classifier']} \n" + f"Number of features: {best_result['#features']}\n" + "Features:" + str(best_result['features']) +"\n" + \
f"Best Params: {best_result['best_params']}" +"\n" + f"Best AUC: {best_result['best_auc']:.4f}" +"\n" + f"Best Sens: {best_result['best_sensitivity']:.4f}" +"\n"+ \
f"Best Spec: {best_result['best_specificity']:.4f}" + "\n"  + f"Best Accuracy: {best_result['best_accuracy']:.4f}" + "\n"


# Create the LaTeX table as a string
latex_table = f"""
\\begin{{table}}[htbp]
\\centering
\\caption{{Best Classifier Performance and Selected Features}}
\\begin{{tabular}}{{|l|l|}}
\\hline
\\textbf{{Best Classifier}} & {best_result['classifier']} \\\\
\\hline
\\textbf{{Classifier Parameters}} & {best_result['best_params']} \\\\
\\hline
\\textbf{{Number of Features}} & {best_result['#features']} \\\\
\\hline
\\textbf{{Feature List}} & {', '.join(best_result['features'])} \\\\
\\hline
\\textbf{{Best AUC}} & {best_result['best_auc']:.2f} $\\pm$ {best_result['best_std_auc']:.2f} \\\\
\\hline
\\textbf{{Best Sensitivity}} & {best_result['best_sensitivity']:.2f} $\\pm$ {best_result['best_std_sensitivity']:.2f} \\\\
\\hline
\\textbf{{Best Specificity}} & {best_result['best_specificity']:.2f} $\\pm$ {best_result['best_std_specificity']:.2f} \\\\
\\hline
\\textbf{{Best Accuracy}} & {best_result['best_accuracy']:.2f} $\\pm$ {best_result['best_std_accuracy']:.2f} \\\\
\\hline
\\end{{tabular}}
\\label{{tab:best_classifier}}
\\end{{table}}
"""
print(best)
#print(latex_table)

with open(f'{tw}tw_smote.txt', 'w') as f:
    f.write(latex_table)


2
LogisticRegression
FS
auc:  0.77
RandomForestClassifier
FS
auc:  0.655
XGBgClassifier
FS
auc:  0.6533333333333333
SVC
FS
auc:  0.7533333333333333
NB
FS
auc:  0.7033333333333334
DTClassifier
FS
auc:  0.58
3
LogisticRegression
FS
auc:  0.8133333333333332
RandomForestClassifier
FS
auc:  0.5933333333333334
XGBgClassifier
FS
auc:  0.6366666666666667
SVC
FS
auc:  0.78
NB
FS
auc:  0.7166666666666666
DTClassifier
FS
auc:  0.6
4
LogisticRegression
FS
auc:  0.7066666666666667
RandomForestClassifier
FS
auc:  0.6766666666666667
XGBgClassifier
FS
auc:  0.63
SVC
FS
auc:  0.7233333333333334
NB
FS
auc:  0.65
DTClassifier
FS
auc:  0.7
5
LogisticRegression
FS
auc:  0.7066666666666667
RandomForestClassifier
FS
auc:  0.6433333333333333
XGBgClassifier
FS
auc:  0.635
SVC
FS
auc:  0.7133333333333333
NB
FS
auc:  0.6633333333333333
DTClassifier
FS
auc:  0.6833333333333333
6
LogisticRegression
FS
auc:  0.5933333333333334
RandomForestClassifier
FS
auc:  0.7466666666666667
XGBgClassifier
FS
auc:  0.626666666666

In [60]:
# test the best models in the Coimbra'a Cohort
for k in range(2,6):
    print(k)
    train_set = pd.read_csv('../data/lisbon/conversion_ad/{}tw_no_norm.csv'.format(k))
    y_train = train_set['Evolution'].copy()

    #train_set, test_set, y_train, y_test = train_test_split(train_set, y_train, test_size=0.20, random_state=42, stratify=y_train)
    test_set = pd.read_csv('../data/coimbra/conversion_ad/{}tw_no_norm.csv'.format(k))
    y_test = test_set['Evolution'].copy()

    if k == 2:
        clf = SVC(C=10, kernel = 'poly', probability = True, random_state = 42)
        features = ['COCA1', 'TPP1', 'HPT', 'CBLN1', 'ACYP2', 'GDIA', 'DHPR', 'HBB', 'TIMP2', 'SGCE', 'NPTXR', 'SODM', 'AFAM']
    elif k == 3:
        clf = XGBClassifier(learning_rate = 0.01, max_depth =5, n_estimators = 50, objective = 'binary:logistic', random_state = 42)
        features = ['HBB', 'HRG', 'IBP6', 'ACYP2']
    elif k == 4:
        clf =GaussianNB(var_smoothing=10**-9)
        features = ['RELN', 'CNTN2', 'CLC11', 'NRP2', 'CNTP4', 'APOL1', 'EPDR1', 'TAGL', 'APOA1', 'TRFE', 'HPT', 'EGFLA', 'CSPG2', 'CIP4', 'EFNB3', 'idade', 'GRIA4', 'NRX1A', 'LFNG', 'ACTS', 'CBLN4', 'NEUS']
    elif k == 5:
        clf =RandomForestClassifier(n_estimators= 10, random_state=42)
        features = ['APOL1', 'NRP2', 'CIP4', 'TRFE', 'RELN', 'CLC11', 'SORL', 'APOA1', 'HPT', 'C163A', 'GRIA4', 'APLP2', 
                    'PRELP', 'FGFR2', 'CBLN4', 'IGA2', 'PLXB2', 'TAGL', 'CNTN6', 'IBP6', 'SPON1', 'SAP', 'MMSE', 'SAA4', 'NPC2',
                    'ALBU', 'FRIL', 'CNTN2', 'SGCE', 'EFNB3', 'C1RL', 'CNTP4', 'EGFLA', 'MYO6', 'EPDR1', 'VGF', 'idade', 'CAHD1', 
                    'NRX1A', 'PTPRF', 'MANBA', 'NEUS', 'RTN4R', 'A2AP', 'NAR3', 'TIMP2', 'KLKB1', 'AMBP', 'DHPR', 'KV127', 'TPP1', 
                    'NEO1', 'CPVL', 'B4GA1', 'COMP', 'HBA', 'LFNG', 'APLP1', 'ALS', 'OMD', 'CSPG2', 'NPTXR', 'A1AG1', 'NID2', 'NCAN', 
                    'FUCO2', 'KPYM', 'ROBO1', 'PEDF', 'FETUA', 'MA1C1', 'AEBP1', 'FGFR1', 'AFAM', 'CFAI', 'HRG', 'SAP3', 'ANT3', 'FHR1', 
                    'PGS2', 'FUCO', 'HEMO', 'SODM', 'CADH2', 'COCA1', 'IGK', 'FBLN2', 'TTHY', 'LTBP1', 'NBL1', 'KV106', 'LDHB', 'HV349',
                    'ACTS', 'CBPQ', 'PEBP1', 'LV657', 'APOB', 'KV37', 'C4BPA', 'LTBP4', 'CFAB', 'FIBG', 'ITIH4', 'CSFdataabeta42', 'CO4B', 
                    'APOA4', 'C1QA', 'RNAS4', 'COFA1', 'CO9', 'PPIC', 'DKK3', 'ICAM5', 'SLIK1', 'CNTP2', 'CERU', 'CSFdatatTau', 'PON1', 'CO6A2',
                    'VWF', 'H6ST3', 'GDIA', 'CBLN1', 'PTPRZ', 'A1BG', 'ITIH1', 'HV118', 'KNG1', 'C1QT3', 'HEXB', 'LV310', 'NID1', 'A1AG2', 
                    'CSFdatapTau', 'PCDC2', 'C1QT1', 'SLIK4', 'FSTL5', 'SPRL1', 'A2GL', 'ICOSL', 'CO6', 'ZA2G', 'HBB', 'LV39',
                    'CBPB2', 'ENDD1', 'DCC', 'KCC2A', 'C1QC', 'CBG', 'MDHC', 'ACYP2']

    train_set = train_set[features]
    test_set = test_set[features]

    scaler = MinMaxScaler()
    train_set = scaler.fit_transform(train_set)
    test_set = scaler.transform(test_set)

    smote = SMOTE(k_neighbors=3, sampling_strategy=0.8, random_state = 42)
    train_set, y_train = smote.fit_resample(train_set, y_train)

    clf.fit(train_set, y_train)
    y_pred = clf.predict(test_set)
    auc = roc_auc_score(y_test, clf.predict_proba(test_set)[:,1])
    sensitivity = sens(y_test, y_pred)
    specificity = spec(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)
    results_str = f'AUC: {auc} \n Sensitivity: {sensitivity} \n Specificity: {specificity} \n Accuracy: {accuracy} \n'
    print(results_str)
    latex_table = f"""
        \\begin{{table}}[htbp]
        \\centering
        \\caption{{Best Classifier Performance and Selected Features {k}tw}}
        \\begin{{tabular}}{{|l|l|}}
        \\hline
        \\textbf{{AUC}} & {auc:.2f}\\
        \\hline
        \\textbf{{Sensitivity}} & {sensitivity:.2f} \\
        \\hline
        \\textbf{{Specificity}} & {specificity:.2f} \\
        \\hline
        \\textbf{{Accuracy}} & {accuracy:.2f} \\
        \\hline
        \\end{{tabular}}
        \\label{{tab:best_classifier}}
        \\end{{table}}
        """
    with open(f'{k}tw_smote_coimbra_test.txt', 'w') as f:
        f.write(latex_table)
    
    

2
AUC: 0.43956043956043966 
 Sensitivity: 0.07692307692307693 
 Specificity: 0.5714285714285714 
 Accuracy: 0.25 

3
AUC: 0.34375 
 Sensitivity: 0.0625 
 Specificity: 0.75 
 Accuracy: 0.2 

4
AUC: 0.4166666666666667 
 Sensitivity: 0.7777777777777778 
 Specificity: 0.0 
 Accuracy: 0.7 

5
AUC: 0.15789473684210525 
 Sensitivity: 0.9473684210526315 
 Specificity: 0.0 
 Accuracy: 0.9 



# tw 4

Best Classifier: SVC

Number of features: 24

Features:  ['RELN', 'NPTXR', 'CLC11', 'CNTN2', 'NRP2', 'EPDR1', 'CBLN4', 'CNTP4', 'idade', 'EFNB3', 'APOL1', 'TAGL', 'GRIA4', 'LFNG', 'NRX1A', 'ITIH1', 'CADH2', 'CSPG2', 'PEBP1', 'PLXB2', 'HRG', 'CIP4', 'LV657', 'NEUS']

Best Params: {'classification__C': 1, 'classification__kernel': 'rbf', 'classification__probability': True}

Best AUC: 0.7517

Best Sens: 0.9300

Best Spec: 0.1900

Best Accuracy: 0.7000


SMOTE: Best Classifier: SVC
Number of features: 49
Features:['CLC11', 'NPTXR', 'RELN', 'CNTP4', 'CNTN2', 'LFNG', 'CBLN4', 'EFNB3', 'HRG', 'PEBP1', 'CIP4', 'APOL1', 'NRP2', 'PLXB2', 'LV657', 'TAGL', 'HPT', 'GRIA4', 'HBB', 'CSPG2', 'FGFR2', 'SLIK1', 'idade', 'EGFLA', 'KCC2A', 'OMD', 'MYO6', 'COMP', 'PGS2', 'CADH2', 'EPDR1', 'HBA', 'SODM', 'NRX1A', 'NEUS', 'CBG', 'ROBO1', 'ITIH1', 'IGA2', 'SCG1', 'KV37', 'APOB', 'CSFdataabeta42', 'MMSE', 'ANT3', 'ENDD1', 'A1AG1', 'TPP1', 'VWF']
Best Params: {'classification__C': 10, 'classification__kernel': 'linear', 'classification__probability': True}
Best AUC: 0.7367
Best Sens: 0.7867
Best Spec: 0.4400
Best Accuracy: 0.6767